# Data Analysis and Visualization Assignment-1

## Team Memebers (NAME + ROLL)
-   Chirag - 12140520
-   Anant Yadav -12140190

# Data Scraping + EDA + Clustering

In [ ]:
%pip install beautifulsoup4 selenium webdriver_manager tqdm

Note: you may need to restart the kernel to use updated packages.


## Import Libraries

In [ ]:
import time
import os
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import lxml

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


## WebDriver Creation

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
# Driver creation
chrome_options = Options() # Initializing am options object

#chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
#chrome_options.add_argument("--incognito")
#chrome_options.binary_location = "/sbin/brave" # optional, if you don't have chrome

# Silent download of drivers
os.environ['WDM_LOG'] = 'False'

# Create service
webdriver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = webdriver_service, options = chrome_options)

## Data Scraping
Scrape the job-postings for the following roles in India:
- Data Engineer
- Data Analyst
- Data Scientist
- Machine Learning Engineer

**Instructions**
- Scrape 300 Job postings per role
- For each job posting, extract the following information:
     - job_title
     - job_role (one of the above 4)
     - company
     - job_location
     - job_summary
     - post_date (posted how many days ago)
     - today (date)
     - job_salary(if not present, fill it with NaN)
     - Job_url
- Create a dataframe from this information

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime

In [ ]:
Total_jobs=[]
roles = ['Data Engineer', 'Data Analyst', 'Data Scientist', 'Machine Learning Engineer']
# Scrape 300 Job postings per role

for role in roles:
    print(f"Companies offering jobs for {role} role:-")
    cnt=0
    for i in range(30):
        cnt+=1
        print(cnt)
        link = f"https://in.indeed.com/jobs?q={role.replace(' ', '+')}&l=India&start={10*i}"
        driver.get(link)
        page_source = driver.page_source
        # Parse 1-page
        soup = BeautifulSoup(page_source, 'lxml')
        all_jobs = soup.find_all('div', attrs = {'class':'job_seen_beacon'})
        for jobs in all_jobs:
            dict_job={}
            try:
                dict_job['title']=jobs.find('h2', attrs = {'class' : 'jobTitle css-1h4a4n5 eu4oa1w0'}).text
                dict_job['job_role']=role
                dict_job['companyName']=jobs.find('span', attrs = {'class' : 'companyName'}).text
                dict_job['job_location']=jobs.find('div', attrs = {'class' : 'companyLocation'}).text
                dict_job['Job_Summary']=jobs.find('div',attrs={'class':'job-snippet'}).text.strip()
                dict_job['post_date']=jobs.find('span',attrs={'class':'date'}).text
                salary=jobs.find('div', attrs = {'class' : 'metadata salary-snippet-container'})
                if salary:
                    dict_job['salary']=salary.text
                else:
                    dict_job['salary']=float("NaN")

                dict_job['today_date']=datetime.today().strftime('%Y-%m-%d')

                dict_job['Job_url']=jobs.find('a',attrs={'class' : 'jcs-JobTitle css-jspxzf eu4oa1w0'})['href']
            except Exception as e:
                Errors=e

            Total_jobs.append(dict_job)
        time.sleep(3)

Companies offering jobs for Data Engineer role:-
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Companies offering jobs for Data Analyst role:-
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Companies offering jobs for Data Scientist role:-
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Companies offering jobs for Machine Learning Engineer role:-
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [ ]:
Total_jobs

[{'title': 'Software Engineer 2- BIOS Automation',
  'job_role': 'Data Engineer',
  'companyName': 'Dell Technologies',
  'job_location': 'Bengaluru, Karnataka',
  'Job_Summary': 'As a Software Engineer 2, responsible for the design and development activities associated with methods for Automating /Validating hardware/software systems,…',
  'post_date': 'PostedPosted 30+ days ago',
  'salary': nan,
  'today_date': '2023-03-15',
  'Job_url': '/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364Be5JbcCJKJ8S-TlvC_iwEgvLx09tUBe2prqhCP1yYc_3luScx6p_JXsY0uz1l0w6PbcYvBD42XJDFLVI-vtm1ivCkXHiCB0sNdowjXep6r1uXWziOJGJmLoPVjBHQk8qXs4T3uc7iDgMcUp8lYAGxD30fI7IzIvkyO2nIJS2GvatCft02JCG_FAhnBym8oiRdYVgizraN2d74Ri3TfxEZW8EuCCfpN7ACr8VIFAHw6qoNczUwkqpOC_UOsteUe1ULdR5w0eEqZiJ4iNeeKuq8yZa1wPv3TSgQSc0VJSA3D8xk1RPTCyLmGwMW7vLsWpxt35sns0RpJHiC3JAkIhgZguJ77NA9_GQmR5shHMobAYLmbWNOogf6IX9ZKmcJ6Z-HhpCvbsPkeKfY3Zza8P720t73ssQm6wqOBvYsukufQIp9n5DHerz__m2ExNSMsKpUMJzUotcU9Ac1Kfd7tbZb-vNvStM1pLFBQXYj0sVeamQeInqsg5ghNfNMAoab

In [ ]:
df = pd.DataFrame(Total_jobs)

In [ ]:
df.to_csv('Indeed.csv', index=False)

## Exploratory Data Analysis (EDA)

### Data Cleaning
- Clean the data.
- Calculate the date of posting from the columns('post_data', 'today')
- Split the 'job_salary' column into 'job_salary_lower' and 'job_salary_upper'
- Create a new column called 'is_remote', which will be true if its a remote job.
- Clean the 'job_location' column to include only city name in India, if no city is found, make it by default 'India'. If its remote make the value as 'Remote'

### Visualization
- How are the jobs distributed geographically?
- Show which companies have more than 5 job-postings
- Show the distribution of number of characters in description.
- Show the distribution of salaries in remote jobs based on roles.
- Show the distribution of salaries in 'Bangalore'

### Data Cleaning

#### Clean the Data

In [ ]:
df = pd.read_csv('Indeed.csv')
df.describe()

,title,job_role,companyName,job_location,Job_Summary,post_date,salary,today_date,Job_url
count,1200,1200,1200,1200,1200,1200,1200,1200,1200
unique,536,4,775,140,1001,42,222,1,1053
top,Data Scientist,Data Engineer,Mercedes-Benz Research and Development India...,"Bengaluru, Karnataka",Software Engineer-SAP Abap Adobe Forms.\nAs a ...,PostedPosted 30+ days ago,?,2023-03-15,/company/NOVEL-FINANCIALS-BUSINESS-SOLUTIONS-P...
freq,103,300,27,356,22,731,839,1200,4


In [ ]:
df['salary'].fillna('?', inplace=True)
df['title'].fillna('?', inplace=True)
df

,title,job_role,companyName,job_location,Job_Summary,post_date,salary,today_date,Job_url
0,Software Engineer 2- BIOS Automation,Data Engineer,Dell Technologies,"Bengaluru, Karnataka","As a Software Engineer 2, responsible for the ...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...
1,Broadcast Engineer,Data Engineer,"Conference Technologies, Inc.","Pune, Maharashtra",Key participant in business development includ...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0AiNMvsi1d0NtjZYR...
2,Software Engineer,Data Engineer,Clarivate,"Bengaluru, Karnataka",The team consists of 10-12 engineers and is re...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BUkGQXKvz_vqTVnr...
3,SR Mulesoft Engineer,Data Engineer,Giant Eagle,"Yelahanka Satellite Town, Karnataka",Work on problems of moderate scope where analy...,PostedPosted 13 days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0B9Z5kUrYpJSl1jY-...
4,R216181 Manager 2 Software Engineering - Embed...,Data Engineer,Dell Technologies,"Bengaluru, Karnataka","As a Software Engineering Manager 2, leading a...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...
...,...,...,...,...,...,...,...,...,...
1195,Senior Machine Learning Engineer,Machine Learning Engineer,BlackBerry,"Noida, Uttar Pradesh",3+ years of experience designing and building ...,PostedPosted 21 days ago,?,2023-03-15,/rc/clk?jk=b6aa6c5de1e7b004&fccid=a311d3f78a94...
1196,Speech Engineer,Machine Learning Engineer,Gnani Innovations,"Bengaluru, Karnataka","Good understanding of signal processing, machi...",PostedPosted 30+ days ago,?,2023-03-15,/rc/clk?jk=a5257efd3c2a9bd7&fccid=1ef8a80b6513...
1197,Data Engineer,Machine Learning Engineer,Barclays,"Pune, Maharashtra",Demonstrated hands-on experience delivering a ...,PostedPosted 30+ days ago,?,2023-03-15,/rc/clk?jk=ec5e9aa9bb18d0cf&fccid=057abf3fd357...
1198,ML Engineer,Machine Learning Engineer,GWC,"Bengaluru, Karnataka",Optimize and fine-tune machine learning models...,PostedPosted 19 days ago,?,2023-03-15,/company/GWC/jobs/Ml-Engineer-21d6d5c828bbaec0...


In [ ]:
df1 = df.copy()
df1.drop(df1.index[(df1["title"] == "?")],axis=0, inplace=True)
df1

,title,job_role,companyName,job_location,Job_Summary,post_date,salary,today_date,Job_url
0,Software Engineer 2- BIOS Automation,Data Engineer,Dell Technologies,"Bengaluru, Karnataka","As a Software Engineer 2, responsible for the ...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...
1,Broadcast Engineer,Data Engineer,"Conference Technologies, Inc.","Pune, Maharashtra",Key participant in business development includ...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0AiNMvsi1d0NtjZYR...
2,Software Engineer,Data Engineer,Clarivate,"Bengaluru, Karnataka",The team consists of 10-12 engineers and is re...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BUkGQXKvz_vqTVnr...
3,SR Mulesoft Engineer,Data Engineer,Giant Eagle,"Yelahanka Satellite Town, Karnataka",Work on problems of moderate scope where analy...,PostedPosted 13 days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0B9Z5kUrYpJSl1jY-...
4,R216181 Manager 2 Software Engineering - Embed...,Data Engineer,Dell Technologies,"Bengaluru, Karnataka","As a Software Engineering Manager 2, leading a...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...
...,...,...,...,...,...,...,...,...,...
1195,Senior Machine Learning Engineer,Machine Learning Engineer,BlackBerry,"Noida, Uttar Pradesh",3+ years of experience designing and building ...,PostedPosted 21 days ago,?,2023-03-15,/rc/clk?jk=b6aa6c5de1e7b004&fccid=a311d3f78a94...
1196,Speech Engineer,Machine Learning Engineer,Gnani Innovations,"Bengaluru, Karnataka","Good understanding of signal processing, machi...",PostedPosted 30+ days ago,?,2023-03-15,/rc/clk?jk=a5257efd3c2a9bd7&fccid=1ef8a80b6513...
1197,Data Engineer,Machine Learning Engineer,Barclays,"Pune, Maharashtra",Demonstrated hands-on experience delivering a ...,PostedPosted 30+ days ago,?,2023-03-15,/rc/clk?jk=ec5e9aa9bb18d0cf&fccid=057abf3fd357...
1198,ML Engineer,Machine Learning Engineer,GWC,"Bengaluru, Karnataka",Optimize and fine-tune machine learning models...,PostedPosted 19 days ago,?,2023-03-15,/company/GWC/jobs/Ml-Engineer-21d6d5c828bbaec0...


In [ ]:
df1.reset_index(inplace = True)
df1 = df1.drop(['index'], axis = 1)
df1.to_csv('Cleaned_Data.csv', index = False)

In [ ]:
df_MLE = df1.loc[df1['job_role'] == 'Machine Learning Engineer']
df_MLE.reset_index(inplace = True)
df_MLE = df_MLE.drop(['index'], axis = 1).loc[:299]
df_MLE.to_csv('Cleaned_Machine_Learning_Engineer.csv')

In [ ]:
df_DS = df1.loc[df1['job_role'] == 'Data Scientist']
df_DS.reset_index(inplace = True)
df_DS = df_DS.drop(['index'], axis = 1).loc[:299]
df_DS.to_csv('Cleaned_Data_Scientist.csv')

In [ ]:
df_DA = df1.loc[df1['job_role'] == 'Data Analyst']
df_DA.reset_index(inplace = True)
df_DA = df_DA.drop(['index'], axis = 1).loc[:299]
df_DA.to_csv('Cleaned_Data_Analyst.csv')

In [ ]:
df1.loc[df1['job_role'] == 'Data Engineer'].loc[:299].to_csv('Cleaned_Data_Engineer.csv', index = False)
df_DE = pd.read_csv('Cleaned_Data_Engineer.csv')

In [ ]:
final_data = pd.concat([df_DE, df_DA, df_DS, df_MLE], ignore_index=True)
final_data.to_csv('Final_Data.csv', index = False)

In [ ]:
df1 = pd.read_csv('Final_Data.csv')

#### Calculate the date of posting from the columns ('post_data', 'today')

In [ ]:
#Add Date of Posting
df_date = df1
df_date['post_date'].unique()
from datetime import timedelta
inp = ['Posted', 'ongoing', 'Active']
def delay(x):
    if 'Posted' in x:
        numb1 = [30 for k in x.split() if (k[0:2] == '30')]
        numb2 = [int(k) for k in x.split() if k.isdigit()]
        numb = numb1 + numb2 + [0]
    if 'Active' in x:
        numb1 = [30 for k in x.split() if (k[0:2] == '30')]
        numb2 = [int(k) for k in x.split() if k.isdigit()]
        numb = numb1 + numb2
    if 'ongoing' in x:
        numb = [0]
    return(numb[0])
def delay_date(date, days):
    subtracted_date = pd.to_datetime(date) - timedelta(days=days)
    subtracted_date = subtracted_date.strftime("%Y-%m-%d")
    return subtracted_date
df_date['date of posting'] = df_date['post_date'].apply(lambda x: delay_date('2023-03-15', delay(x)))
df1 = df_date

#### Split the 'job_salary' column into 'job_salary_lower' and 'job_salary_upper'

In [ ]:
#Add job_salary_lower and job_salary_upper
df_sal = df1
from numpy import NaN
def change_Salary(Salaries):
  dict_salary={'job_salary_lower':[],"job_salary_upper":[]}
  for x in Salaries:
    if x=='?':
      x='?'
      dict_salary['job_salary_lower'].append('?')
      dict_salary['job_salary_upper'].append('?')
    elif(x[0]=='₹' and x[-4:]=='year') and '-' in x:
      dict_salary['job_salary_lower'].append(int(x.replace("-","").replace("₹","").split()[0].replace(",","")))
      dict_salary['job_salary_upper'].append(int(x.replace("-","").replace("₹","").split()[1].replace(",","")))

    elif(x[0]=='₹' and x[-4:]=='hour') and '-' in x:
      dict_salary['job_salary_lower'].append(int(x.replace("-","").replace("₹","").split()[0].replace(",",""))*8*220)
      dict_salary['job_salary_upper'].append(int(x.replace("-","").replace("₹","").split()[1].replace(",",""))*8*220)

    elif(x[0]=='₹' and x[-4:]=='year'):
      dict_salary['job_salary_lower'].append(float(NaN))
      dict_salary['job_salary_upper'].append(x.replace("-","").replace("₹","").split()[0].replace(",",""))

    elif(x[0]=='₹' and x[-4:]=='hour'):
      dict_salary['job_salary_lower'].append(float(NaN))
      dict_salary['job_salary_upper'].append(int(x.replace("₹","").split()[0].replace(",",""))*8*220)

    elif(x[0]=='₹' and x[-3:]=='day'):
      dict_salary['job_salary_lower'].append(int(x.replace("-","").replace("₹","").split()[0].replace(",",""))*220)
      dict_salary['job_salary_upper'].append(int(x.replace("-","").replace("₹","").split()[1].replace(",",""))*220)

    elif(x[:4]=='From' and x[-4:]=='year'):
      dict_salary['job_salary_lower'].append(x.split()[1].replace(",","").replace("₹",""))
      dict_salary['job_salary_upper'].append(float(NaN))

    elif(x[:4]=='From' and x[-3:]=='day'):
      dict_salary['job_salary_lower'].append(int(x.split()[1].replace(",","").replace("₹",""))*220)
      dict_salary['job_salary_upper'].append(float(NaN))

    elif(x[:4]=='From' and x[-4:]=='hour'):
      dict_salary['job_salary_lower'].append(int(x.split()[1].replace(",","").replace("₹",""))*8*220)
      dict_salary['job_salary_upper'].append(float(NaN))

    elif(x[0]=='₹' and x[-5:]=='month'):
      dict_salary['job_salary_lower'].append(float(NaN))
      dict_salary['job_salary_upper'].append(int(x.split()[0].replace(",","").replace("₹",""))*12)

    elif x[:2]=='Up':
      dict_salary['job_salary_lower'].append(float(NaN))
      dict_salary['job_salary_upper'].append(int(x.replace("₹","").split()[2].replace(",","")))
    elif x[:4]=='From' and x[-5:]=='month':
      dict_salary['job_salary_lower'].append(int(x.replace("₹","").split()[1].replace(",",""))*12)
      dict_salary['job_salary_upper'].append(float(NaN))
    else:
      print(x)
      break
  return dict_salary
dict_saly_upp_low=change_Salary(df_sal['salary'])
job_salary_lower = list(dict_saly_upp_low['job_salary_lower'])
job_salary_upper = list(dict_saly_upp_low['job_salary_upper'])
df_sal['job_salary_lower'] = job_salary_lower
df_sal['job_salary_upper'] = job_salary_upper
df_sal.head()
df1 = df_sal

#### Create a new column called 'is_remote', which will be true if it is a remote job

In [ ]:
#Add is_remote
df_remote = df1
def remote(x):
    if 'Remote' in x:
        return True
    else:
        return False
df_remote['is_remote'] = df_remote['job_location'].apply(lambda x: remote(x))
df1 = df_remote

#### Clean the 'job_location' column to include only city name in India, if no city is found, make it by default 'India'. If it's remote, make the value as 'Remote'

In [ ]:
#Updated Job Location
df_city = df1
cities=[]
for location in df1['job_location']:
  cities.append(location.split(',')[0])
cities=[city if city[0:6]!='Remote' else "Remote" for city in cities]
states=["Andhra Pradesh","Arunachal Pradesh ","Assam","Bihar","Chhattisgarh","Goa","Gujarat","Haryana","Himachal Pradesh","Jammu and Kashmir","Jharkhand","Karnataka","Kerala","Madhya Pradesh","Maharashtra","Manipur","Meghalaya","Mizoram","Nagaland","Odisha","Punjab","Rajasthan","Sikkim","Tamil Nadu","Telangana","Tripura","Uttar Pradesh","Uttarakhand","West Bengal","Andaman and Nicobar Islands","Chandigarh","Dadra and Nagar Haveli","Daman and Diu","Lakshadweep","National Capital Territory of Delhi","Puducherry"]
for index,city in enumerate(cities):
  if city in states:
    cities[index]="India"
df_city['job_location'] = cities
df1 = df_city

### Visualization

#### How are the jobs distributed geographically?

In [ ]:
px.bar(df1,x='job_location',color='job_role',title='Geographic Distribution')

In [ ]:
def returnstate(x):
  states=["Andhra Pradesh","Arunachal Pradesh ","Assam","Bihar","Chhattisgarh","Goa","Gujarat","Haryana","Himachal Pradesh","Jammu and Kashmir","Jharkhand","Karnataka","Kerala","Madhya Pradesh","Maharashtra","Manipur","Meghalaya","Mizoram","Nagaland","Odisha","Punjab","Rajasthan","Sikkim","Tamil Nadu","Telangana","Tripura","Uttar Pradesh","Uttarakhand","West Bengal","Andaman and Nicobar Islands","Chandigarh","Dadra and Nagar Haveli","Daman and Diu","Lakshadweep","National Capital Territory of Delhi","Puducherry"]
  for state in states:
    if state in x:
      return state
df2=df[['title','job_location','job_role']]
df2.reset_index()
new_df=df2['job_location'].apply(lambda x: returnstate(x)).reset_index()
df2.loc[:,['job_location']]=new_df.loc[:,['job_location']]
fig=px.bar(df2,x='job_location',color='job_role',title='Geographic Distribution Statewise')
fig.show()

<ipython-input-48-7813c22738d5>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def count_dict(column):
  states=["Andhra Pradesh","Arunachal Pradesh ","Assam","Bihar","Chhattisgarh","Goa","Gujarat","Haryana","Himachal Pradesh","Jammu and Kashmir","Jharkhand","Karnataka","Kerala","Madhya Pradesh","Maharashtra","Manipur","Meghalaya","Mizoram","Nagaland","Odisha","Punjab","Rajasthan","Sikkim","Tamil Nadu","Telangana","Tripura","Uttar Pradesh","Uttarakhand","West Bengal","Andaman and Nicobar Islands","Chandigarh","Dadra and Nagar Haveli","Daman and Diu","Lakshadweep","National Capital Territory of Delhi","Puducherry"]
  state_count={f"{state}":0 for state in states}
  for state in states:
    for values in column:
      if state in values:
        state_count[state]+=1
  return state_count
state_count=count_dict(df['job_location'])
x=list(state_count.keys())
y=list(state_count.values())
statewisejob={'States':x,'Number_of_Jobs':y}
fig = plt.figure(figsize = (30, 5))
fig=px.bar(statewisejob, x='States', y='Number_of_Jobs',title='Geographic distributions of Jobs Statewise')
fig.show()

<Figure size 2160x360 with 0 Axes>

#### Show which companies have more than 5 job-postings

In [ ]:
#List of Companies having more than 5 job postings
df_jobPosting = df1.groupby('companyName')['title'].count()
df_jobPosting = pd.DataFrame(df_jobPosting).reset_index()

In [ ]:
print(list(df_jobPosting['companyName'][df_jobPosting['title'] > 5]))

['ARA Resources', 'Accenture', 'Airtel India', 'Barclays', 'Bechtel', 'Bosch Group', 'Boston Consulting Group', 'Cardinal Health', 'DXC Technology', 'Deloitte', 'Deutsche Bank', 'Genpact', 'Infosys Limited', 'Mercedes-Benz Research and Development India...', 'PayPal', 'PayU', 'Telstra', 'Uber']


In [ ]:
print(df_jobPosting.head())
px.bar(df_jobPosting,x='companyName',y='title',title="Comapnies offering more than 5 job postings")

                    companyName  title
0                   12th Wonder      1
1     3Minds eSolutions Pvt Ltd      1
2                       5x Data      1
3    8andH Engineering Services      2
4  A for Analytics IT Solutions      2


#### Show the distribution of number of characters in description

In [ ]:
desc_dist=df1[['job_role','Job_Summary']]
desc_dist['total_characters']=desc_dist['Job_Summary'].apply(lambda x : len(x))

<ipython-input-23-3a01b84c848d>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
desc_dist=desc_dist.reset_index()
desc_dist.head()

,index,job_role,Job_Summary,total_characters
0,0,Data Engineer,"As a Software Engineer 2, responsible for the ...",158
1,1,Data Engineer,Key participant in business development includ...,132
2,2,Data Engineer,The team consists of 10-12 engineers and is re...,160
3,3,Data Engineer,Work on problems of moderate scope where analy...,112
4,4,Data Engineer,"As a Software Engineering Manager 2, leading a...",158


In [ ]:
fig = px.scatter(desc_dist, y="total_characters", x="index",color='job_role')
fig.show()

#### Show the distribution of salaries in remote jobs based on roles

In [ ]:
#Updated Salary (removed NaN and '?' for Remote Jobs)
salary_job=df1[df1['job_location']=='Remote'][['job_role','job_salary_lower','job_salary_upper']]
update_Salary_job=salary_job[(salary_job['job_salary_lower']!='?') & (salary_job['job_salary_upper']!='?')]
for ind in update_Salary_job.index:
  if pd.isnull(update_Salary_job['job_salary_lower'][ind]):
    update_Salary_job['job_salary_lower'][ind]=update_Salary_job['job_salary_upper'][ind]
  if pd.isnull(update_Salary_job['job_salary_upper'][ind]):
    update_Salary_job['job_salary_upper'][ind]=update_Salary_job['job_salary_lower'][ind]

<ipython-input-26-45b7d9782167>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-45b7d9782167>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
data_11 = (update_Salary_job['job_salary_lower'][df1['job_role'] == 'Data Engineer']).to_numpy()
data_12 = (update_Salary_job['job_salary_lower'][df1['job_role'] == 'Data Analyst']).to_numpy()
data_13 = (update_Salary_job['job_salary_lower'][df1['job_role'] == 'Data Scientist']).to_numpy()
data_14 = (update_Salary_job['job_salary_lower'][df1['job_role'] == 'Machine Learning Engineer']).to_numpy()
data_21 = (update_Salary_job['job_salary_upper'][df1['job_role'] == 'Data Engineer']).to_numpy()
data_22 = (update_Salary_job['job_salary_upper'][df1['job_role'] == 'Data Analyst']).to_numpy()
data_23 = (update_Salary_job['job_salary_upper'][df1['job_role'] == 'Data Scientist']).to_numpy()
data_24 = (update_Salary_job['job_salary_upper'][df1['job_role'] == 'Machine Learning Engineer']).to_numpy()

In [ ]:
df_low_sal =pd.DataFrame(dict(
    series=np.concatenate((["Data Engineer"]*len(data_11), ["Data Analyst"]*len(data_12), ["Data Scientist"]*len(data_13), ["Machine Learning Engineer"]*len(data_14))),
    Salary_Band  =np.concatenate((data_11,data_12, data_13, data_14))))
px.histogram(df_low_sal, x="Salary_Band", color="series", barmode="overlay",  nbins=32, title = 'Lower Salary')

In [ ]:
df_up_sal =pd.DataFrame(dict(
    series=np.concatenate((["Data Engineer"]*len(data_21), ["Data Analyst"]*len(data_22), ["Data Scientist"]*len(data_23), ["Machine Learning Engineer"]*len(data_24))),
    Salary_Band  =np.concatenate((data_21,data_22, data_23, data_24))))
px.histogram(df_up_sal, x="Salary_Band", color="series", barmode="overlay",  nbins=32, title = 'Upper Salary')

#### Show the distribution of salaries in 'Banglore'

In [ ]:
#Updated Salary (removed NaN and '?' for 'Bengaluru')
salary_bang=df1[df1['job_location']=='Bengaluru'][['job_role','job_salary_lower','job_salary_upper']]
update_Salary_Bang=salary_bang[(salary_bang['job_salary_lower']!='?') & (salary_bang['job_salary_upper']!='?')]
for ind in update_Salary_Bang.index:
  if pd.isnull(update_Salary_Bang['job_salary_lower'][ind]):
    update_Salary_Bang['job_salary_lower'][ind]=update_Salary_Bang['job_salary_upper'][ind]
  if pd.isnull(update_Salary_Bang['job_salary_upper'][ind]):
    update_Salary_Bang['job_salary_upper'][ind]=update_Salary_Bang['job_salary_lower'][ind]
update_Salary_Bang['job_salary_lower'] = pd.to_numeric(update_Salary_Bang['job_salary_lower'])
update_Salary_Bang['job_salary_upper'] = pd.to_numeric(update_Salary_Bang['job_salary_upper'])

<ipython-input-30-e1d731f457c2>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-e1d731f457c2>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-e1d731f457c2>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-e1d731f457c2>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [ ]:
mean_sal_Banglore = []
for i in range(len(update_Salary_Bang['job_salary_lower'])):
    mean_sal_Banglore.append(list(update_Salary_Bang['job_salary_lower'])[i] + list(update_Salary_Bang['job_salary_upper'])[i])
px.histogram(x = mean_sal_Banglore, nbins = 50, title = 'Salary Distribution in Banglore')

## Clustering

Perform the following type of clustering of salaries for 4 roles(Data Engineer,Data Analyst,Data
Scientist,Machine Learning Engineer) separately for different regions in India.
1. K-Means/K-Medians
2. Agglomerative Clustering

Visualize the clusters and answer some questions:
1. Is the salary of Data Scientist in Bangalore higher than Hyderabad?
2. Which location should I favour for ML Engineer Role?


In [ ]:
columns=df1.columns
dfsal=df1[columns]

In [ ]:
df1['job_location'].value_counts()

Bengaluru          363
Remote             244
India               77
Pune                70
Hyderabad           68
                  ... 
Panaji               1
Medchal              1
Wakad                1
Powai                1
Electronic City      1
Name: job_location, Length: 83, dtype: int64

In [ ]:
df1.head()

,title,job_role,companyName,job_location,Job_Summary,post_date,salary,today_date,Job_url,date of posting,job_salary_lower,job_salary_upper,is_remote
0,Software Engineer 2- BIOS Automation,Data Engineer,Dell Technologies,Bengaluru,"As a Software Engineer 2, responsible for the ...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...,2023-02-13,?,?,False
1,Broadcast Engineer,Data Engineer,"Conference Technologies, Inc.",Pune,Key participant in business development includ...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0AiNMvsi1d0NtjZYR...,2023-02-13,?,?,False
2,Software Engineer,Data Engineer,Clarivate,Bengaluru,The team consists of 10-12 engineers and is re...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BUkGQXKvz_vqTVnr...,2023-02-13,?,?,False
3,SR Mulesoft Engineer,Data Engineer,Giant Eagle,Yelahanka Satellite Town,Work on problems of moderate scope where analy...,PostedPosted 13 days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0B9Z5kUrYpJSl1jY-...,2023-03-02,?,?,False
4,R216181 Manager 2 Software Engineering - Embed...,Data Engineer,Dell Technologies,Bengaluru,"As a Software Engineering Manager 2, leading a...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...,2023-02-13,?,?,False


In [ ]:
columns=df1.columns
new_df=df1[columns]


In [ ]:
new_df

In [ ]:
new_df.head()

,title,job_role,companyName,job_location,Job_Summary,post_date,salary,today_date,Job_url,date of posting,job_salary_lower,job_salary_upper,is_remote
0,Software Engineer 2- BIOS Automation,Data Engineer,Dell Technologies,Bengaluru,"As a Software Engineer 2, responsible for the ...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...,2023-02-13,?,?,False
1,Broadcast Engineer,Data Engineer,"Conference Technologies, Inc.",Pune,Key participant in business development includ...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0AiNMvsi1d0NtjZYR...,2023-02-13,?,?,False
2,Software Engineer,Data Engineer,Clarivate,Bengaluru,The team consists of 10-12 engineers and is re...,PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BUkGQXKvz_vqTVnr...,2023-02-13,?,?,False
3,SR Mulesoft Engineer,Data Engineer,Giant Eagle,Yelahanka Satellite Town,Work on problems of moderate scope where analy...,PostedPosted 13 days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0B9Z5kUrYpJSl1jY-...,2023-03-02,?,?,False
4,R216181 Manager 2 Software Engineering - Embed...,Data Engineer,Dell Technologies,Bengaluru,"As a Software Engineering Manager 2, leading a...",PostedPosted 30+ days ago,?,2023-03-15,/pagead/clk?mo=r&ad=-6NYlbfkN0BdeCH3wzlYpD364B...,2023-02-13,?,?,False
